### Shared Word

In [81]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize

In [82]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [83]:
lang_folder = "Turkish"  # Primary language
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

In [84]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [85]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_And_English (eng)_Etymologeek_Result_All.xlsx']

In [86]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,şu,şut,Turkish,(Noun) (football) shoot.(football) shoot.,shoot,English (eng),(mining) A vein of ore running in the same gen...
1,alo,alo,Turkish,(Interjection) Hello (only when picking up the...,heaven,English (eng),"(Christianity, Islam) The afterlife of the sou..."
2,tür,türban,Turkish,(Noun) Turban (man’s and women’s head-dress).T...,turban,English (eng),A man's headdress made by winding a length of ...
3,çek,çek,Turkish,"(Noun) Check, cheque.Check, cheque.",cheque,English (eng),"(Australia, Canada, India, New Zealand, UK) A ..."
4,film,film,Turkish,(Noun) A medium used to capture images in a ca...,film,English (eng),"(cinema, uncountable) Film or movies as a grou..."
5,test,test,Turkish,(Noun) Test.Test.,test,English (eng),(obsolete) A witness. (obsolete) To make a tes...
6,seksi,seksi,Turkish,(Adjective) Sexy.Sexy.,sex,English (eng),"(chiefly, US, colloquial, intransitive) To hav..."
7,lord,lord,Turkish,(Noun) Lord.Lord.,lord,English (eng),"(intransitive, and, transitive) Domineer or ac..."
8,taksi,taksi,Turkish,(Noun) Taxi.Taxi.,taximeter,English (eng),A device installed in a taxicab that calculate...
9,futbol,futbol,Turkish,(Noun) Football (soccer).Football (soccer).,foot,English (eng),(countable) A biological structure found in ma...


In [71]:
word_list = df_pair["dict_entry_main"].values.tolist()

In [115]:
# Select N Gram Type
sent_ngram_file = "Five" # Two, Three, Four, Five, Sentence
 
if sent_ngram_file.lower() != "sentence":
    data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
    folder_path = "N Gram"
else:
    data_kind = "sentence"
    folder_path = "Sentence"

In [116]:
df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
#df = df.head(100000)
df

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [117]:
d_list  = df.iloc[:,0].values.tolist()

In [118]:
resultlist = []

In [119]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(d_list):
    mergelist = []
    try:
        word = d_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(d_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list) # string_word liste

In [120]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
df_merge_result.drop_duplicates(inplace=True)
df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
df_merge_result.reset_index(drop=True, inplace=True)
df_merge_result

,fivegram,frequency


In [121]:
df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)

#### Concat Result

In [122]:
df_word = df_pair.loc[:,["dict_entry_main"]]
df_word.rename(columns={"dict_entry_main":"word"}, inplace=True)
df_word

,word
0,şut
1,alo
2,türban
3,çek
4,film
5,test
6,seksi
7,lord
8,taksi
9,futbol


In [123]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,futbol basketbol,26
1,şut çek,25
2,alo radyo,20
3,lider futbol,12
4,alo sos,12
5,lider sosis,12
6,maç boks,12
7,tenis futbol,10
8,striptiz seksi,10
9,basketbol futbol,9


In [124]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram


In [125]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram


In [126]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram


In [127]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,şut çek,39
1,seksi kovboy,9


In [128]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,şut,futbol basketbol,26.0,NaN,NaN,NaN,NaN,NaN,NaN,şut çek,39.0
1,alo,şut çek,25.0,NaN,NaN,NaN,NaN,NaN,NaN,seksi kovboy,9.0
2,türban,alo radyo,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,çek,lider futbol,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,film,alo sos,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,test,lider sosis,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,seksi,maç boks,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,lord,tenis futbol,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,taksi,striptiz seksi,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,futbol,basketbol futbol,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result.xlsx", index=False)